This notebook walks through the process of identifying shoreline from Planet imagery using Otsu thresholding of the normalized differenced water index (NDWI), as was done for the following publication: https://doi.org/10.5194/egusphere-2024-1656 . A few notes:

- The user will need to provide their own imagery
- This code requires matplotlib version 3.7.2 or lower
- The process is not fully automated. The user will need to manually iterate through images and select the appropriate countour for each one
- Planet Scences are sometimes delivered as mutiple files. These will need to be merged and the merged image added to the image file directory prior to running this notebook. We used the QGIS merge function to merge a scene from  
25 June 2019.
  
contact: m1bryant@ucsd.edu

In [1]:
import rasterio
from rasterio.transform import from_gcps
from rasterio.control import GroundControlPoint as GCP
import numpy as np
import matplotlib.pyplot as plt #IMPORTANT should be version 3.7.2 or lower
import pyproj
from matplotlib import cm
from affine import Affine
import os
import glob
from shapely import wkt
import datetime
import shutil
import sys

In [26]:
#Helper functions:
def trim_image(A, X, Y, xs, ys):
    #A: 3D array containing image data
    #X, Y: 2D arrays of full image coordinates
   #xs, ys: tuples containing desired bounds for trimming
    x = X[0, :]
    y = Y[:, 0]
    x_t = x[(x >= xs[0]) & (x <= xs[1])] 
    y_t = y[(y >= ys[0]) & (y <= ys[1])]
 
    x1 = np.where(x == x_t[0])[0][0]
    x2 = np.where(x == x_t[len(x_t)-1])[0][0]
    y2 = np.where(y == y_t[len(y_t)-1])[0][0]
    y1 = np.where(y == y_t[0])[0][0]
    A_small = A[:, y1:y2, x1:x2]
    X_small = X[y1:y2, x1:x2]
    Y_small = Y[y1:y2, x1:x2]
    return A_small, X_small, Y_small

def read_file(image_file, lons= None, lats = None):
    with rasterio.open(image_file) as f:
        T0 = f.transform  # upper-left pixel corner affine transform
        p = (f.crs) #projection (UTM in this case)
        A = f.read()
    n_rows = A.shape[1]
    n_cols = A.shape[2]
# Get affine transform for pixel centres
    T1 = T0 * Affine.translation(0.5, 0.5)

# convert row/column index to eastings and norhtings

    min_coords = T1*(0, 0)
    max_coords = T1*(n_cols, n_rows)
    x_min = min_coords[0]
    x_max = max_coords[0]
    y_min = min_coords[1]
    y_max = max_coords[1]
    x = np.linspace(x_min, x_max, n_cols)
    y = np.linspace(y_min, y_max, n_rows)
    X, Y = np.meshgrid(x, y)
    if lons != None:
        A, X, Y = trim_image(A, X, Y, lons, lats)
    return A,p, X, Y 

def make_rgb(A):
    #A: 3D array containing image data
    blue = np.float64(A[0, :, :])
    green = np.float64(A[1, :, :])
    red = np.float64(A[2, :, :])
    
    
    #re-assign very bringt (snow/ice) pixels
    blue[np.where(blue > 3000)] = 3000
    red[np.where(red > 3000)] = 3000
    green[np.where(green > 3000)] = 3000
    #contrast enhancement
    red_n = ((red - np.min(red[red >0]))*1/(np.max(red) - np.min(red[red >0])))
    blue_n = ((blue - np.min(blue[blue > 0]))*1/(np.max(blue) - np.min(blue[blue>0])))
    green_n = ((green - np.min(green[green > 0]))*1/(np.max(green) - np.min(green[green > 0])))



    rgb_image = np.stack([red_n, green_n, blue_n], axis = 2).reshape((red.shape[0], red.shape[1], 3))
    return rgb_image

def get_ndwi(A):
    green = np.float64(A[1, :, :])
    ir = np.float64(A[3, :, :])
    ndwi = (ir-green)/(ir+green)
    return ndwi

def otsu_thresholding(img, n_bins):
    hist_n, bins= np.histogram(img[~np.isnan(img)], n_bins, density = True) #normalized histogram
    hist, bins= np.histogram(img[~np.isnan(img)], n_bins) #histogram with counts
    t = 0 #arbitrrily starting threshold
    var = 1e-6 #arbitrailiy small starting variance
    var_list = []
    for b in np.arange(len(bins)-1):
        t_new = bins[b+1] #right edge of highest bin (our potential threshold value)
        hist_1 = hist_n[0:b]
        bins_1 = bins[1:b+1]
        bins_2 = bins[b+1:len(bins)]
        hist_2 = hist_n[b:len(hist_n)]
        q1 = np.sum(hist_1) #sum of probabilities below threshold
        q2 = np.sum(hist_2) #sum of probabilities above threshold
        #get means and variancesof the two catgories
        mu_1 = np.sum(hist_1*bins_1/q1)
        mu_2 = np.sum(hist_2*bins_2/q2)
    
        #calculate weighted between-class variance (we want to maximize this value)
        var_t = (q1*q2)*(mu_1-mu_2)**2
        var_list.append(var_t)
        if var_t > var:
            t = t_new
            var = var_t 
    return t

def date_from_fname(file): 
    datestring = file[0:8]
    return datestring

def id_from_fname(file):
    segments = file.split('_')
    f_id = [s for s in segments if len(s)==4][0]
    return f_id


def get_contours(file, lons, lats):
    A,p, X, Y = read_file(file, lons, lats)
    rgb = make_rgb(A)
    
    fig_hist = plt.figure()
    image = get_ndwi(A)
    t = otsu_thresholding(image, 100)
    plt.hist(image.flatten(), bins = 100)
    plt.xlabel('NDWI', fontsize = 15)
    plt.ylabel('Frequency', fontsize = 15)
    plt.axvline(t, c = 'k', ls = '--')
    
    fig_0 = plt.figure()
    c = plt.contour(X, Y, image, levels = [t])
    plt.close(fig_0)
    contours = c.allsegs[0]
    #sort by length
    l = [int(len(i)) for i in contours]
    l_s = [int(len(i)) for i in contours]
    l_s.sort(reverse=True)
    indices = l_s[0:10]
    fig = plt.figure()
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.imshow(rgb, extent = (np.min(X), np.max(X), np.min(Y), np.max(Y)))
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.imshow(image, extent = (np.min(X), np.max(X), np.min(Y), np.max(Y)), cmap = cm.gray)
    contour_list = [contours[np.where(l == np.int64(i))[0][0]] for i in indices][0:10] #save the 10 longest
    for i in range(0, len(contour_list)):
        ax1.plot(contour_list[i][:, 0], contour_list[i][:, 1], c = cm.tab10(i))#c = cm.tab10(i)
        ax2.plot(contour_list[i][:, 0], contour_list[i][:, 1], c = 'tab:blue')
    sm = plt.cm.ScalarMappable(cmap=cm.tab10, norm=plt.Normalize(vmin=0, vmax=10))
    plt.colorbar(sm, ax = ax1)
    plt.colorbar(sm, ax = ax2)
    
    return contour_list

def save_countour(output_dir, contour, date, fid):
    os.chdir(output_dir)
    np.savetxt("coastline_" + date + '_' + f_id + '_ndwi' + '.csv', contour, delimiter=",")

In [20]:
data_dir = 'data/Planet Images' #replace with appropriate data directory
os.chdir(data_dir)

##bbox for study area
lons = [469400, 477700]

lats = [7864300, 7865200]

output_dir = '/Users/m1bryant/Documents/Python/tc_paper_code/data/coastlines/all_coastlines' #replace with appropriate directory
files = glob.glob('*SR_clip.tif')
files.sort()



# Iterating through files 

The next 2 code blocks provide code to run outsu thresholding on an image, plot and label all of the contours that could be a shoreline, and save the specified contour. In order to iterate through all imagery and save successful coastline aquisitions:

- increase 'file_num' to go to the next image and run the first cell 
- Inspect the output. The legend on the contour plot shows the index label for each contour
- If you want to save one of the plotted contours, set  'c' in the second cell to the corresponding index and run the cell

In [33]:
%matplotlib auto
os.chdir(data_dir)
file_num = 3 #update this number to iterate through directory
file = files[file_num]
date = date_from_fname(file)
f_id = id_from_fname(file)
contour_list =  get_contours(file, lons, lats)    
plt.suptitle(date + ' ' + f_id)



Using matplotlib backend: MacOSX


Text(0.5, 0.98, '20190625 1006')

In [25]:
c = 0

save_countour(output_dir, contour_list[c], date, f_id)